In [27]:
import nltk
import ssl
import pandas as pd
from operator import itemgetter
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
import string
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [28]:
train = pd.read_csv('Corona_NLP_train.csv',encoding='latin-1',usecols=['OriginalTweet','Sentiment'])
test = pd.read_csv('Corona_NLP_test.csv',encoding='latin-1',usecols=['OriginalTweet','Sentiment'])

In [29]:
train.head()

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [30]:
# A
# Pre-processing data

for row in range(len(train)):
    if train.at[row,"Sentiment"] == "Extremely Negative":
        train.at[row,"Sentiment"] = "Negative"
    elif train.at[row,"Sentiment"] == "Extremely Positive":
        train.at[row,"Sentiment"] = "Positive"
    elif train.at[row,"Sentiment"] == "Neutral":
        train = train.drop(row)
        
for row in range(len(test)):
    if test.at[row,"Sentiment"] == "Extremely Negative":
        test.at[row,"Sentiment"] = "Negative"
    elif test.at[row,"Sentiment"] == "Extremely Positive":
        test.at[row,"Sentiment"] = "Positive"
    elif test.at[row,"Sentiment"] == "Neutral":
        test = test.drop(row)

In [31]:
# Check and reset indices
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
train.head()

,OriginalTweet,Sentiment
0,advice Talk to your neighbours family to excha...,Positive
1,Coronavirus Australia: Woolworths to give elde...,Positive
2,My food stock is not the only one which is emp...,Positive
3,"Me, ready to go at supermarket during the #COV...",Negative
4,As news of the regionÂs first confirmed COVID...,Positive


In [32]:
train.describe()

,OriginalTweet,Sentiment
count,33444,33444
unique,33444,2
top,advice Talk to your neighbours family to excha...,Positive
freq,1,18046


In [33]:
# B
# Need to remove URLS and numbers using regex
# Sub from when we see https until the next space. Needs to be https followed by maybe :, ;, ''
# \S matches any char that is not whitespace
# \d+ matches digits

for row in range(len(train)):
    train.at[row,"OriginalTweet"] = re.sub(r'https?\S+', '',   train.at[row,"OriginalTweet"])
    train.at[row,"OriginalTweet"] = re.sub(r'\d+', '',   train.at[row,"OriginalTweet"])
        
for row in range(len(test)):
    test.at[row,"OriginalTweet"] = re.sub(r'https?:\S+', '',   test.at[row,"OriginalTweet"])
    test.at[row,"OriginalTweet"] = re.sub(r'\d+', '',   test.at[row,"OriginalTweet"])

In [34]:
# C tokenize

for row in range(len(train)):
    train.at[row,"OriginalTweet"] = nltk.word_tokenize(train.at[row,"OriginalTweet"])

for row in range(len(test)):
    test.at[row,"OriginalTweet"] = nltk.word_tokenize(test.at[row,"OriginalTweet"])

In [35]:
print(train.at[1,"OriginalTweet"][1])

Australia


In [36]:
# turn to lowercase

for row in range(len(train)):
    for word in range(len(train.at[row,"OriginalTweet"])):
        train.at[row,"OriginalTweet"][word] =  (train.at[row,"OriginalTweet"][word]).lower()
            
for row in range(len(test)):
    for word in range(len(test.at[row,"OriginalTweet"])):
        test.at[row,"OriginalTweet"][word] =  (test.at[row,"OriginalTweet"][word]).lower()

In [37]:
train.head()

,OriginalTweet,Sentiment
0,"[advice, talk, to, your, neighbours, family, t...",Positive
1,"[coronavirus, australia, :, woolworths, to, gi...",Positive
2,"[my, food, stock, is, not, the, only, one, whi...",Positive
3,"[me, ,, ready, to, go, at, supermarket, during...",Negative
4,"[as, news, of, the, regionâs, first, confirme...",Positive


In [38]:
# D, remove stopwords, punctuation, stem words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

for row in range(len(train)):
    train.at[row,"OriginalTweet"] = [word for word in train.at[row,"OriginalTweet"] if word not in stop_words and word not in string.punctuation]
    for word in range(len(train.at[row,"OriginalTweet"])):
        train.at[row,"OriginalTweet"][word] = nltk.PorterStemmer().stem(train.at[row,"OriginalTweet"][word])

for row in range(len(test)):
    test.at[row,"OriginalTweet"] = [word for word in test.at[row,"OriginalTweet"] if word not in stop_words and word not in string.punctuation]
    for word in range(len(test.at[row,"OriginalTweet"])):
        test.at[row,"OriginalTweet"][word] = nltk.PorterStemmer().stem(test.at[row,"OriginalTweet"][word])

In [54]:
# E,F combine the sets to get BoW
 
# What is the purpose of combining the test and train if we need to split again anyways...?
corpus = []
trainBoW = []
testBoW = []

for row in range(len(train)):
    corpus.append(" ".join(train.at[row,"OriginalTweet"]))
    trainBoW.append(" ".join(train.at[row,"OriginalTweet"]))
    
for row in range(len(test)):
    corpus.append(" ".join(test.at[row,"OriginalTweet"]))
    testBoW.append(" ".join(test.at[row,"OriginalTweet"]))

In [56]:
# G
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer=None, stop_words=None)

BoW = vectorizer.fit_transform(corpus)
trainBoW = BoW[:len(train)]
testBoW = BoW[len(train):]

In [57]:
# G, Train model
from sklearn.naive_bayes import MultinomialNB

MNB = MultinomialNB()
MNB.fit(trainBoW, train["Sentiment"].values)

MultinomialNB()

In [58]:
# H, Evaluate
from sklearn import metrics
predicted = MNB.predict(testBoW)
accuracy_score = metrics.accuracy_score(predicted, test["Sentiment"].values)

In [60]:
print("The accuracy of the model is: " + str(accuracy_score))

The accuracy of the model is: 0.780119534444794
